In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import timeit
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn import datasets

In [3]:
## Coreset Algorithm
clusters = {}
centroids = []
np_data = []
total_for_coreset = 0
#function to generate 'num_points' random points of 'dim' dimensions.
def generate_data(data, m):
	dim = data.shape[1]
	rows = data.shape[0]
	data_mean = np.mean(data, axis = 0)
	distance = 0
	for point in data:
		distance += np.sum(np.square(point-data_mean))
	prob_dist = []
	#calculating proposal  distribution for each row
	for point in data:
		value=((0.5*(1/n))+0.5*((np.sum(np.square(point-data_mean)))/distance))
		prob_dist.append(value)
	df = pd.DataFrame(prob_dist)
	data = pd.DataFrame(data)
	data['Prob_dist'] = df
	weight_value = []
	for i in range(n):
		weight = 1/(m*prob_dist[i])
		weight_value.append(weight)
	df = pd.DataFrame(weight_value)
	data['weight_value'] = df
	sampled_points = data.sample(n=m, frac=None, replace=False, weights='weight_value', random_state=None, axis=None)
	return sampled_points.iloc[:,:dim]

In [4]:
data = pd.read_csv('smartphone.csv',sep=",")
data = np.array(data)
data = data[:,:data.shape[1]-1]
# Identifying data length
n = data.shape[0]
k = 6
## Finding the number of attributes.
features = data.shape[1]
start = timeit.default_timer()
points = np.array(generate_data(data, k))
stop = timeit.default_timer()
total_for_coreset = stop-start

In [5]:
## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k, init = points, n_init =1).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start

In [6]:
# No. of. Nearest Neighbors
n_n = 7

## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## generating Query point
query = []
for i in range(0,features):
    x = round(random.random()*random.choice([1,10,-1,-10]),random.choice([1,2,3,4,5]))
    query.append(x)

query = np.array(query)
query = query.reshape(1,features)


## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
stop = timeit.default_timer()   
time_for_closest_center = stop-start   
## Finding the cluster elements to construct KD-Tree

cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values)
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(total_for_coreset,time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(total_for_coreset + time_for_kmeans + time_for_closest_center + time_for_kdtree + query_time)

[[ 0.142    0.0608  -0.119   ... -0.811    0.226   -0.00751]
 [ 0.402   -0.00845 -0.135   ... -0.857    0.194   -0.00906]
 [ 0.204   -0.00697 -0.111   ... -0.877    0.182    0.00641]
 ...
 [ 0.26    -0.0297  -0.0947  ... -0.871    0.187    0.0188 ]
 [ 0.296    0.0181  -0.114   ... -0.858    0.196    0.00716]
 [ 0.201   -0.0308  -0.0752  ... -0.851    0.201    0.0255 ]] [101.68697735 101.76717046 101.80321572 101.83875743 101.90668418
 101.91168066 101.92063642]
0.7550939139991897 0.7467593409992332 0.003900210999745468 0.006836399999883724 0.0008875390003595385
1.5134774049984117


In [7]:
start = timeit.default_timer()
tree = KDTree(data)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
j=0
start = timeit.default_timer()
dist_dum , ind_dum = tree.query(query,n_n)
stop = timeit.default_timer()
total_time = stop-start
print(total_time)
for i in ind_dum:
    print(data[i],dist_dum[j])
    j =+ 1

0.40204199600066204
0.011570181999559281
[[ 1.42e-01  6.08e-02 -1.19e-01 ... -8.11e-01  2.26e-01 -7.51e-03]
 [ 2.92e-01 -2.05e-02 -7.99e-02 ... -7.51e-01  2.66e-01  5.30e-02]
 [ 4.02e-01 -8.45e-03 -1.35e-01 ... -8.57e-01  1.94e-01 -9.06e-03]
 ...
 [ 2.04e-01 -6.97e-03 -1.11e-01 ... -8.77e-01  1.82e-01  6.41e-03]
 [ 2.58e-01 -7.00e-05 -1.40e-01 ... -7.60e-01  2.51e-01 -4.69e-02]
 [ 2.89e-01  3.88e-02 -6.79e-02 ... -7.29e-01  2.79e-01  6.75e-02]] [101.68697735 101.72181046 101.76717046 101.79943039 101.80321572
 101.8125171  101.83770697]


In [8]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
import random
import timeit

# Loading data
data = pd.read_csv('smartphone.csv',sep=",")
data = np.array(data)
data = data[:,:data.shape[1]-1]
# Identifying data length
n = data.shape[0]
k = 6

# No. of. Nearest Neighbors
n_n = 7

## Finding the number of attributes.
features = data.shape[1]


## Running KMeans on train data.
start = timeit.default_timer()
y_predict = KMeans(n_clusters = k).fit(data)
stop = timeit.default_timer()
time_for_kmeans = stop-start
## Storing Cluster centers in cluster_centers
cluster_centers = np.array(y_predict.cluster_centers_)

## Finding the nearest cluster to query point.
    ## Finding the euclidean distance from first cluster_center point.
start = timeit.default_timer()
dist = 0
for i in range(features):
   dist += (query[0,i]-cluster_centers[0,i])**2
dist = np.sqrt(dist)
near_cluster = 0
i = 1

 ## Finding the euclidean distance from other cluster_center points.
while i < k:
    dist_j = 0
    for j in range(features):
       dist_j += (query[0,j]-cluster_centers[i,j])**2
    dist_j = np.sqrt(dist_j)
    if(dist_j < dist):
        dist = dist_j
        near_cluster = i
    i += 1
    
stop = timeit.default_timer()   
time_for_closest_center = stop-start
## Finding the cluster elements to construct KD-Tree
cluster_values = np.ones([n,features])
j = 0
for i in range(n):
    if y_predict.labels_[i] == near_cluster:
        cluster_values[j] = data[i]
        j += 1
cluster_values = cluster_values[:j,:j]

## KDTree Construction and Querying
start = timeit.default_timer()
tree = KDTree(cluster_values, np.log(cluster_values.shape[0]))
stop = timeit.default_timer()
time_for_kdtree = stop-start

start = timeit.default_timer()
dist_main , ind_main = tree.query(query[0:1],n_n)
stop = timeit.default_timer()
query_time = stop-start


j=0
for i in ind_main:
    print(cluster_values[i], dist_main[j])
    j += 1
print(time_for_kmeans,time_for_closest_center,time_for_kdtree,query_time)
print(time_for_closest_center + time_for_kdtree + query_time)

[[ 0.142    0.0608  -0.119   ... -0.811    0.226   -0.00751]
 [ 0.402   -0.00845 -0.135   ... -0.857    0.194   -0.00906]
 [ 0.204   -0.00697 -0.111   ... -0.877    0.182    0.00641]
 ...
 [ 0.26    -0.0297  -0.0947  ... -0.871    0.187    0.0188 ]
 [ 0.296    0.0181  -0.114   ... -0.858    0.196    0.00716]
 [ 0.201   -0.0308  -0.0752  ... -0.851    0.201    0.0255 ]] [101.68697735 101.76717046 101.80321572 101.83875743 101.90668418
 101.91168066 101.92063642]
5.235056831999827 0.004167363999840745 0.010726420000537473 0.0016939859997364692
0.016587770000114688
